# 量子门与量子线路基础操作

In [1]:
# pylint: disable=W0104
import sys
import mindspore as ms

path = '../'
sys.path.append(path)                                # 添加自主开发的量子模拟器代码所在路径
ms.set_context(mode=ms.PYNATIVE_MODE)                # 使用动态图
# ms.set_context(mode=ms.GRAPH_MODE)

In [2]:
import numpy as np
from qusmoke.gates import H, X, Y, Z, RX, RY, RZ, CNOT, ZZ, SWAP, U1, U2, U3
from qusmoke.circuit import Circuit

In [3]:
# 构建一个基本的线路，线路中的每个参数门均为 mindspore.nn.Cell 的子类

circ = Circuit([
    H(0),
    X(0, [2,3]),
    Y(1),
    Z(1, 2),
    RX('param').on(3),
    RY(1.0).on(2, 3),
    ZZ(2.0).on((0, 1)),
    SWAP((1, 3)),
    U3(1.0, 2.0, 3.0).on(0),
    U3(1.0, 2.0, 3.0).on(1),
    U3(1.0, 2.0, 3.0).on(2),
    U3(1.0, 2.0, 3.0).on(3),
    X(1, [0, 2, 3]),
    RY(2.0).on(3, [0, 1, 2])
])

print(circ)

Circuit<
  (gates): SequentialCell<
    (0): H<0>
    (1): X<0, [2, 3]>
    (2): Y<1>
    (3): Z<1, 2>
    (4): RX(param)<3>
    (5): RY(_param_)<2, 3>
    (6): ZZ(_param_)<(0, 1)>
    (7): SWAP<(1, 3)>
    (8): U3(u3_params)<0>
    (9): U3(u3_params)<1>
    (10): U3(u3_params)<2>
    (11): U3(u3_params)<3>
    (12): X<1, [0, 2, 3]>
    (13): RY(_param_)<3, [0, 1, 2]>
    >
  >


In [4]:
# 其中输出 '_param_' 为默认参数名
circ.params_name

['param',
 '_param_',
 '_param_',
 'u3_params',
 'u3_params',
 'u3_params',
 'u3_params',
 '_param_']

In [5]:
# 支持线路等加法运算
circ += RZ('param_z').on(3)
circ += [X(2), Y(0, 3)]
circ += Circuit([Z(1), Z(2)])

print(circ)

Circuit<
  (gates): SequentialCell<
    (0): H<0>
    (1): X<0, [2, 3]>
    (2): Y<1>
    (3): Z<1, 2>
    (4): RX(param)<3>
    (5): RY(_param_)<2, 3>
    (6): ZZ(_param_)<(0, 1)>
    (7): SWAP<(1, 3)>
    (8): U3(u3_params)<0>
    (9): U3(u3_params)<1>
    (10): U3(u3_params)<2>
    (11): U3(u3_params)<3>
    (12): X<1, [0, 2, 3]>
    (13): RY(_param_)<3, [0, 1, 2]>
    (14): RZ(param_z)<3>
    (15): X<2>
    (16): Y<0, 3>
    (17): Z<1>
    (18): Z<2>
    >
  >


In [6]:
# 打印线路信息

circ.summary()

=================================================Circuit Summary=================================================
|Total number of gates  : 19.                                                                                   |
|Parameter gates        : 9.                                                                                    |
|with 9 parameters are  :                                                                                       |
|param, _param_, _param_, u3_params, u3_params, u3_params, u3_params, _param_, param_z                        . |
|Number qubit of circuit: 4                                                                                     |


In [7]:
# 获取线路状态

circ.get_qs()

array([ 0.08451916-0.05577248j,  0.16271482+0.0295833j ,
       -0.00641707-0.04917871j,  0.17163612-0.24420995j,
        0.16867927+0.05835877j,  0.13320881+0.259294j  ,
        0.13453737+0.00128568j,  0.15733941+0.1533909j ,
        0.14758667+0.01421016j,  0.16689745+0.08064345j,
        0.03495108+0.08881146j,  0.07131908-0.05617252j,
        0.28291166-0.45243284j,  0.06628986+0.31992695j,
        0.09377108-0.3911432j ,  0.12404367+0.21276072j], dtype=complex64)

In [8]:
# 给可变参数赋予特定的值，对初始未给定名字的参数（默认名为 '_param_' ）不支持赋值
qs = circ.get_qs({'param': 1.0, 'param_z': 2.0})

print(f'n_qubit: {circ.n_qubit}')
print(f'qs:\n{qs}')

n_qubit: 4
qs:
[ 0.02713373-0.09186346j  0.13572592-0.07779382j -0.03236606-0.03037797j
 -0.02406999-0.2790749j   0.15023226-0.07741043j  0.2647003 +0.07822535j
  0.13123125-0.07579522j  0.24045055+0.05868228j  0.09202915+0.09560809j
  0.07232455+0.15972522j -0.02893956+0.08921152j  0.0811841 +0.00335759j
  0.46028954-0.2083434j  -0.11549888+0.28698888j  0.40331292-0.20640226j
 -0.09110902+0.26201683j]


In [9]:
# 打印线路可训练参数

circ.trainable_params()

[Parameter (name=4.param, shape=(), dtype=Float32, requires_grad=True),
 Parameter (name=5.param, shape=(), dtype=Float32, requires_grad=True),
 Parameter (name=6.param, shape=(), dtype=Float32, requires_grad=True),
 Parameter (name=8.param, shape=(3,), dtype=Float32, requires_grad=True),
 Parameter (name=9.param, shape=(3,), dtype=Float32, requires_grad=True),
 Parameter (name=10.param, shape=(3,), dtype=Float32, requires_grad=True),
 Parameter (name=11.param, shape=(3,), dtype=Float32, requires_grad=True),
 Parameter (name=13.param, shape=(), dtype=Float32, requires_grad=True),
 Parameter (name=14.param, shape=(), dtype=Float32, requires_grad=True)]

In [10]:
# 将线路变为编码线路

circ.as_encoder()
# 或执行 circ.no_grad()，功能与 circ.as_encoder() 一致
circ.trainable_params()

[]

In [11]:
# 线路状态和 mindquantum 对比
# 需要安装 mindquantum 0.8，可在 CPU 上运行进行对比

import mindquantum as mq
    
mq_circ = mq.Circuit([
    mq.H(0),
    mq.X(0, [2,3]),
    mq.Y(1),
    mq.Z(1, 2),
    mq.RX('param').on(3),
    mq.RY(1.0).on(2, 3),
    mq.ZZ(2.0).on((0, 1)),
    mq.SWAP((1, 3)),
    mq.U3(1.0, 2.0, 3.0).on(0),
    mq.U3(1.0, 2.0, 3.0).on(1),
    mq.U3(1.0, 2.0, 3.0).on(2),
    mq.U3(1.0, 2.0, 3.0).on(3),
    mq.X(1, [0, 2, 3]),
    mq.RY(2.0).on(3, [0, 1, 2]),
    mq.RZ('param_z').on(3),
    mq.X(2),
    mq.Y(0, 3),
    mq.Z(1),
    mq.Z(2)
])

mq_qs = mq_circ.get_qs(pr={'param': 1.0, 'param_z': 2.0})

print(f'n_qubit: {mq_circ.n_qubits}')
print(f'qs:\n{mq_qs}')

n_qubit: 4
qs:
[ 0.02713373-0.09186347j  0.13572591-0.07779382j -0.03236607-0.03037797j
 -0.02406997-0.27907495j  0.15023226-0.07741042j  0.26470028+0.07822533j
  0.13123125-0.07579524j  0.2404506 +0.05868228j  0.09202917+0.0956081j
  0.07232457+0.15972524j -0.02893956+0.08921154j  0.08118412+0.00335757j
  0.46028954-0.20834341j -0.11549886+0.28698891j  0.40331302-0.20640233j
 -0.09110898+0.26201691j]


In [12]:
# 对比可知，自定义的线路与 mindquantum 的输出一致

print(np.isclose(qs, mq_qs))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [13]:
# 说明，sporequantum 中所有门均需要实例化，不支持 mindquantum 中的
# X.on(obj) 操作，使用 X(obj) 代替

X(0)         # OK
# X.on(0)    # NOT OK

X<0>